# This notebook illsutrates how to evaluate the performance of runs

The main function used is evaluate_project from evaluate.py. To use the function 2 main thigs need to be defined:
- which dataset directories to use:
    - `data_dir` = for test set to use 
- which models to evaluate. To supply these **ONE** of 3 options need to be done:
    - `run_dir_list`  = list of pre-defined runs 
    - `project_dir` = a directory that contains subdirectories of models that will be evaluated
    - `wandb_project_name` = project completed in WandB. If using this option must also provide `wandb_dir` = a directory path where the outputs of the project are saved

Optionally output path `output_dir`, filename `output_prefix` can be defined and `batch_size` can be defined. Set batch size to smaller values for bigger models (at the cost of longer duration)

The outputs are:
- csv of the evaluation results wih 2 rows per target of a model corresponding to scaled or raw results (so the csv is made up of *N(models) * N(targets) * 2 rows*


In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../scripts')
import evaluate
import pandas as pd

In [25]:
%%time
# use list of run directories
evaluate.evaluate_project(data_dir='/mnt/1a18a49e-9a31-4dbf-accd-3fb8abbfab2d/shush/4grid_atac/lite/random_chop/i_2048_w_1/',
                          run_dir_list=['/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/'],
                         batch_size=512)

USING PREDEFINED LIST OF RUNS
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/
CPU times: user 1.79 s, sys: 65.9 ms, total: 1.86 s
Wall time: 1.8 s


In [29]:
evaluation_results = pd.read_csv('output/evaluation_results.csv', index_col='index') # get results from default output path
evaluation_results.head(2)

,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,targets,pred type,eval type,bin_size,...,filtN_1,kern_3,l_rate,loss_fn,lr_patience,model_fn,num_epochs,window_size,run_dir,scaling_factors
index,,,,,,,,,,,,,,,,,,,,,
0,1.156767,0.310932,0.338953,0.776809,0.404083,0.582830,GM21381,raw,whole,32,...,128,25,0.001,poisson,6,bpnet,30,2048,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,NaN
1,1.141954,0.288794,0.318816,0.728043,0.621309,0.736962,GM23338,raw,whole,32,...,128,25,0.001,poisson,6,bpnet,30,2048,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,NaN


In [30]:
eval_type = 'whole'
pred_type = 'raw'

raw_whole_results = evaluation_results[(evaluation_results['pred type']==pred_type)&
                                      (evaluation_results['eval type']==eval_type)]
print('Average Pearson\'s r per run')
raw_whole_results.groupby('run_dir').mean()

Average Pearson's r per run


,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,bin_size,filtN_1,kern_3,l_rate,lr_patience,num_epochs,window_size,scaling_factors
run_dir,,,,,,,,,,,,,,
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/,1.002365,0.342307,0.350024,0.77931,0.524944,0.632311,32,128,25,0.001,6,30,2048,NaN


In [31]:
# per cell line pearson r results
raw_whole_results.sort_values('pr_corr')[['targets', 'pr_corr']]

,targets,pr_corr
index,,
4,HG03575,0.358567
13,PC-3,0.362511
0,GM21381,0.404083
7,GM12878,0.410875
6,DND-41,0.429903
10,IMR-90,0.466413
12,Panc1,0.494923
8,A549,0.515570
14,MCF,0.578344
